In [2]:
!pip install torchaudio

     |████████████████████████████████| 2.7MB 43.9MB/s eta 0:00:01
     |████████████████████████████████| 773.1MB 22kB/s 
ERROR: torchvision 0.4.2 has requirement torch==1.3.1, but you'll have torch 1.3.0 which is incompatible.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1


In [1]:
import os
import time
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
# import torchaudio
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

In [2]:
import json
import matplotlib.pyplot as plt

In [3]:
import os
import torchaudio
import moviepy.editor as mp
from torch.utils.data import Dataset, DataLoader
import torchvision
from  keras import backend as K
from keras.models import load_model, Model
import torchvision.transforms as transforms
from PIL import Image

Using TensorFlow backend.


In [4]:
# !pip install --upgrade moviepy

In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [6]:
import torch.nn as nn

import torch
frame_model = torch.hub.load('pytorch/vision:v0.4.2', 'inception_v3', pretrained=True)
frame_model.eval()

class Identity(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x
    
frame_model.fc = Identity()

Using cache found in /home/yusuf/.cache/torch/hub/pytorch_vision_v0.4.2


In [7]:

def customPooling(x):
    target = x[1]
    inputs = x[0]
    maskVal = 0
    #getting the mask by observing the model's inputs
    mask = K.equal(inputs, maskVal)
    mask = K.all(mask, axis=-1, keepdims=True)

    #inverting the mask for getting the valid steps for each sample
    mask = 1 - K.cast(mask, K.floatx())

    #summing the valid steps for each sample
    stepsPerSample = K.sum(mask, axis=1, keepdims=False)

    #applying the mask to the target (to make sure you are summing zeros below)
    target = target * mask

    #calculating the mean of the steps (using our sum of valid steps as averager)
    means = K.sum(target, axis=1, keepdims=False) / stepsPerSample

    return means

model = load_model("saved_model_240_8_32_0.05_1_50_0_0.0001_100_156_2_True_True_fitted_objects.h5", custom_objects={'customPooling': customPooling})
audio_model = Model(inputs = model.input, outputs = model.layers[-3].output)

In [8]:
img_transforms = transforms.Compose([transforms.Resize((229, 229)),
                                 transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [79]:
class VideoDataset(Dataset):
    """
    A rapper class for the Video dataset.
    """

    def __init__(self, file_path, video_paths, frame_model, audio_model, transform = None, limit=None):
        """
        Args:
            file_path(string): path to the audio csv file
            root_dir(string): directory with all the audio folds
            folds: integer corresponding to audio fold number or list of fold number if more than one fold is needed
        """
        self.video_file = pd.read_json(file_path).transpose()
        self.video_file['label'] = le.fit_transform(self.video_file.label.values)
        # self.folds = folds
        # self.video_paths = glob.glob(video_paths + '/*' + str(self.folds) + '/*')
        self.video_paths = video_paths
        self.frame_model = frame_model.eval()
        self.audio_model = audio_model
        self.transform = transform
        self.limit = limit

    def __len__(self):
        return len(os.listdir(self.video_paths))

    def __getitem__(self, idx):

        video_file = self.video_file.index[idx]
        clip = mp.VideoFileClip(os.path.join(self.video_paths, video_file)) #pass in the video file path here
        no_frames = int(np.round(clip.duration)) * int(np.round(clip.fps))
        
        if self.limit:
            frame_embeddings = np.zeros((self.limit, 2048))
        else:
            frame_embeddings = np.zeros((no_frames, 2048))

        for idx, frame in enumerate(clip.iter_frames()):
            if idx == self.limit:
                break
            frame = Image.fromarray(frame)
            if self.transform:
                frame = self.transform(frame)
            frame_embeddings[idx, :] = (self.frame_model(frame.unsqueeze(0)).detach().numpy()).squeeze(0) #detach from gradients

        audio = clip.audio.to_soundarray()

        audio = audio.mean(1, keepdims=True)
        mel_specgram = torchaudio.transforms.MelSpectrogram(sample_rate=44100, n_fft=2048, win_length=2000, hop_length=500, n_mels=240)(torch.tensor(audio).float().T)  # (channel, n_mels, time)
        mel_specgram = np.moveaxis(mel_specgram.detach().numpy(), 1, 2)
        audio_embedding = [self.audio_model.predict(mel_specgram).squeeze()]
        label = self.video_file['label'][idx]
        
        return torch.tensor(frame_embeddings), torch.tensor(audio_embedding), torch.tensor(label).unsqueeze(0)

In [16]:
clip = mp.VideoFileClip("aassnaulhq.mp4") #pass in the video file path here

In [18]:
np.round(clip.duration)

10.0

In [19]:
np.round(clip.fps)

30.0

In [17]:
sample = clip.get_frame(0)
type(sample)
# sample = torch.tensor(sample)
sample = img_transforms(Image.fromarray(sample))
sample.shape

torch.Size([3, 229, 229])

In [19]:
x = frame_model(sample.unsqueeze(0))
x.flatten().shape


torch.Size([2048])

In [ ]:
clip = mp.VideoFileClip(os.path.join(self.video_paths, video_file)) #pass in the video file path here
frame_embeddings = np.zeros((no_frames, 1, 2048))

for idx, frame in enumerate(clip.iter_frames()):
    frame = Image.fromarray(frame)
    if self.transform:
        frame = self.transform(frame)
#                 frame = frame.permute(2,0,1)
#                 print(frame.shape)
    frame_embeddings[idx, :, :] = self.frame_model(frame.unsqueeze(0)).detach().numpy() #detach from gradients


In [32]:
df = pd.read_json("dfdc_train_part_01/dfdc_train_part_1/metadata.json").T
df['n_label'] = le.fit_transform(df.label.values)

In [10]:
for i in enumerate

,label,split,original,n_label
zumqqvixhu.mp4,FAKE,train,hntguogkqd.mp4,0
utdlsqfykm.mp4,FAKE,train,nswtvttxre.mp4,0
mdfndlljvt.mp4,FAKE,train,ptkcmwnfjv.mp4,0
maktypgsfl.mp4,FAKE,train,objgwnmscm.mp4,0
pleqihjpif.mp4,FAKE,train,xrhqtmxlvx.mp4,0
...,...,...,...,...
qjdtgggqym.mp4,REAL,train,NaN,1
jzmzdispyo.mp4,REAL,train,NaN,1
qeaxtxpvyq.mp4,FAKE,train,keioymnobc.mp4,0
bfeewgzrbr.mp4,FAKE,train,qjdtgggqym.mp4,0


In [80]:
dataset = VideoDataset("dfdc_train_part_01/dfdc_train_part_1/metadata.json", "dfdc_train_part_01/dfdc_train_part_1", frame_model, audio_model, img_transforms)

In [81]:
t0 = time.time()
out = next(iter(dataset))
t1 = time.time()

print("Duration is: ", t1-t0)

Duration is:  26.450217962265015


In [84]:
out[0].shape

torch.Size([300, 2048])

In [86]:
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)


In [87]:
for data in train_loader:
    out = data
    break

In [90]:
out[0].shape

torch.Size([2, 300, 2048])

In [37]:
out[0] = out[0].squeeze(2)#.shape

In [17]:
t0 = time.time()

out = train_loader.batch_sampler

t1 = time.time()

print("Duration is: ", t1-t0)

Duration is:  0.00020956993103027344


TypeError: 'BatchSampler' object is not callable

In [24]:
%time out = next(iter(dataset))

300
CPU times: user 2min 7s, sys: 1.48 s, total: 2min 8s
Wall time: 32.4 s


In [74]:
out[1].shape

torch.Size([1, 1, 50])

In [57]:
out[0][-1,:,:]

tensor([[0.0609, 0.1742, 0.3809,  ..., 0.3465, 1.4547, 0.1772]],
       dtype=torch.float64)

In [ ]:
file_path = '/content/UrbanSound8K/metadata/UrbanSound8K.csv'
audio_paths = '/content/UrbanSound8K/audio'

In [ ]:
#Glorot Initialization Function
def init_weights(m):
    if type(m) == nn.Conv1d or type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight.data)

# Model Classes Definition

In [92]:
class DeepFake(nn.Module):
    def __init__(self):
        super(DeepFake, self).__init__()
        self.LSTM1 = nn.LSTM(2048, 2048, 2, dropout=0.5, batch_first=True)
        self.LSTM2 = nn.LSTM(2048, 512, batch_first=True)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(562, 248)
        self.fc2 = nn.Linear(248, 2)
        
    def forward(self, x, y):
        x, _hidden = self.LSTM1(x.float())
        x = self.relu(x)
        h = _hidden[0].mean(0).unsqueeze(0)
        c = _hidden[1].mean(0).unsqueeze(0)
        x, _ = self.LSTM2(x) #, (h, c))
        print(x[:, -1, :].shape)
        print(y.shape)
        combined = torch.cat((x[:, -1, :], y.squeeze(1)), 1)
        x = self.relu(self.fc1(combined))
        x = self.fc2(x)
        return x

net = DeepFake()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-5, weight_decay=1e-6)


In [93]:
out[0].shape

torch.Size([2, 300, 2048])

In [94]:
%time output = net(out[0], out[1])

torch.Size([2, 512])
torch.Size([2, 1, 50])
CPU times: user 31.4 s, sys: 35.9 ms, total: 31.4 s
Wall time: 7.86 s


In [95]:
output.shape

torch.Size([2, 2])

In [60]:
output.shape

torch.Size([1, 2])

In [61]:
loss = criterion(output, torch.tensor(out[-1]).unsqueeze(0))
loss

tensor(0.0063, grad_fn=<NllLossBackward>)

## One-Epoch Train and Test Functions

In [ ]:

def train(model, train_loader, optimizer, criterion):
  model.train()
  train_loss = 0
  train_correct = 0
  for data, label in train_loader:
    data = data.to(device)
    label = label.to(device)
    optimizer.zero_grad()
    out = model(data)
    train_correct += (torch.argmax(out, dim=1).eq_(label).sum()).item()
    loss = criterion(out, label)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
  avg_loss = train_loss/len(train_loader)
  accuracy = train_correct/(len(train_loader.dataset))
  return avg_loss, accuracy

def test(model, test_loader, criterion):
  with torch.no_grad():
    model.eval()
    test_correct = 0
    test_loss = 0
    for data, label in test_loader:
        data = data.to(device)
        label = label.to(device)
        out2 = model(data)
        loss2 = criterion(out2, label)
        test_loss += loss2.item()
        test_correct += (torch.argmax(out2, dim=1).eq_(label).sum()).item()
    avg_loss = test_loss/len(test_loader)
    accuracy = test_correct/len(test_loader.dataset)  
  return avg_loss, accuracy


# Train for all Models and all Folds for 10 epoch each
### Log results to Tensorboard and JSON file

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# print_every = 10
avg_loss = []
# epochs =100
overall_result = {}

model = DeepFake()

# writer_path = "/content/drive/My Drive/AudioTask/{}".format(models_name[model_idx])
comment = "{}".format(models_name[model_idx])
#Tensorboard Logger
writer = SummaryWriter(comment)

train_dataset = AudioDataset(file_path, audio_paths, train_folds)
test_dataset = AudioDataset(file_path, audio_paths, test_folds)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.023)
early_stopping = EarlyStopping(patience=10, verbose=True)

if os.path.isfile('DeepFake_model_checkpoint.tar'):
    print('DeepFake_model_checkpoint.tar found..')
    print('Loading checkpoint..')
    checkpoint = torch.load('DeepFake_model_checkpoint.tar')
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print('Finished loading checkpoint..')
else:
    train_losses = []
    test_losses = []
    training_time = 0
    no_of_steps = 0

epochs = 10
# print_every = 5

train_losses = []
test_losses = []
train_accuracy = []
test_accuracy = []

for i in range(epochs):
    t0 = time.time()
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    t1 = time.time()
    print("Train Loss at {}/{} is {} | Train accuracy:{} | Time:{}".format(i+1, epochs, train_loss, train_acc, t1-t0))

    t0 = time.time()
    test_loss, test_acc = test(model, test_loader, criterion)
    t1 = time.time()
    print("Test Loss at {}/{} is {} | Test accuracy:{} | Time:{}".format(i+1, epochs, train_loss, train_acc, t1-t0))

    train_losses.append(train_loss)
    train_accuracy.append(train_acc)
    test_losses.append(test_loss)
    test_accuracy.append(test_acc)

    writer.add_scalar("Loss/Test", test_loss, i)
    writer.add_scalar("Accuracy/Test", test_acc, i)
    writer.add_scalar("Loss/Train", train_loss, i)
    writer.add_scalar("Accuracy/Train", train_acc, i)


inter_result.append({'train_loss':train_losses, 'train_accuracy':train_accuracy, 'test_loss':test_losses, 'test_accuracy':test_accuracy})

out_filename = '/content/drive/My Drive/AudioTask/result.json'
with open(out_filename, 'w') as f:
    json.dump(inter_result, f)

print("\n\nEnd of training\nLogging results..")
writer.close()


Train Loss at 1/10 is 1.9544590772503483 | Train accuracy:0.31436097114063216 | Time:103.266526222229
Test Loss at 1/10 is 1.9544590772503483 | Test accuracy:0.31436097114063216 | Time:7.592396020889282
Train Loss at 2/10 is 1.7023153287650894 | Train accuracy:0.4071232249198351 | Time:85.67500424385071
Test Loss at 2/10 is 1.7023153287650894 | Test accuracy:0.4071232249198351 | Time:7.012881755828857
Train Loss at 3/10 is 1.5954135899996236 | Train accuracy:0.4445716903344022 | Time:77.00282979011536
Test Loss at 3/10 is 1.5954135899996236 | Test accuracy:0.4445716903344022 | Time:6.974988698959351
Train Loss at 4/10 is 1.5253564406485454 | Train accuracy:0.4705680256527714 | Time:68.96403694152832
Test Loss at 4/10 is 1.5253564406485454 | Test accuracy:0.4705680256527714 | Time:6.9713146686553955
Train Loss at 5/10 is 1.5229286277381173 | Train accuracy:0.4740036646816308 | Time:68.96565890312195
Test Loss at 5/10 is 1.5229286277381173 | Test accuracy:0.4740036646816308 | Time:7.1190

# Results Plots

In [ ]:
def plot(result, title):
  fig, ax = plt.subplots(1, 2)
  fig.set_size_inches(10,5)
  ax[0].set_title('{} Train + Test Losses'.format(title))
  ax[0].plot(result['train_loss'], label="Train")
  ax[0].plot(result['test_loss'], label="Test")
  ax[0].legend()
  ax[1].set_title('{} Train + Test Accuracy'.format(title))
  ax[1].plot(result['train_accuracy'], label="Train")
  ax[1].plot(result['test_accuracy'], label="Test")
  ax[1].legend()
  plt.show()

In [ ]:
with open('results/Output/results.json', 'r') as f:
  results = json.load(f)

In [ ]:
import pandas as pd
df = pd.read_json('metadata.json').transpose()